In [24]:
import gymnasium as gym
import gymnasium_robotics

# PyTorch
import torch
import os
import numpy as np
import tqdm
from stable_baselines3 import SAC
# from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.buffers import ReplayBuffer

In [25]:
env_id = 'FrankaKitchen-v1'
task = 'kettle'
gym.register_envs(gymnasium_robotics)

In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [27]:
n_actions = 9

In [28]:
def flatten_observation(observation):
    if not isinstance(observation, dict):
        return observation
    achieved = observation['achieved_goal'][task].astype(np.float32)
    obs = observation['observation'].astype(np.float32)

    flat_obs = np.concatenate([achieved, obs], dtype=np.float32)
    return flat_obs

def custom_reward(observation):
    achieved = observation['achieved_goal'][task][0:4]
    desired = observation['desired_goal'][task][0:4]
    res = 1.0 - np.linalg.norm(achieved - desired)
    # assert res <= 1.0 and res >= 0.0, "Reward out of range! "+str(res)
    return res   

In [29]:
flat_dim = 59 + 7
obs_low = np.full((flat_dim,), -1e10, dtype=np.float32)
obs_high = np.full((flat_dim,), 1e10, dtype=np.float32)

class FlattenDictWrapper(gym.Wrapper):    
    def __init__(self, env):
        super().__init__(env)
        self.keys = env.observation_space.spaces.keys()
        self.observation_space = gym.spaces.Box(low=obs_low, high=obs_high, shape=(flat_dim,), dtype=np.float32)

    # def observation(self, observation):
    #     return flatten_observation(observation)
    
    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        if reward == 0.0:
            reward = custom_reward(obs)
        else:
            reward = 1000
        obs = flatten_observation(obs)
        return obs, reward, terminated, truncated, info

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        return flatten_observation(obs)

In [30]:
def make_env():
    env = gym.make(env_id, render_mode=None, tasks_to_complete=[task])  # Or your actual task
    env = FlattenDictWrapper(env)
    return env

n_training_envs = 1
env = DummyVecEnv([make_env]*n_training_envs)
eval_env = DummyVecEnv([make_env])

In [ ]:
combos1 = torch.load("combos/combos-lr001-hs0-just-success-10000")
combos2 = torch.load("combos/combos-lr001-hs0-just-success-10000-5000")

In [ ]:
def initialize_buffer(replay_buffer, combos):
    for combo in tqdm.tqdm(combos):
        state = eval_env.reset()
        for combo_step in combo:
            action = np.asarray([combo_step])
            obs, reward, done, info = eval_env.step(action)
            replay_buffer.add(state[0], obs[0], action, reward, done, info)
            state = obs

In [ ]:
model = SAC("MlpPolicy", env, device=device, learning_rate=0.001)

In [ ]:

replay_buffer = model.replay_buffer
initialize_buffer(replay_buffer, combos1)
initialize_buffer(replay_buffer, combos2)

100%|██████████| 636/636 [06:18<00:00,  1.68it/s]


In [ ]:
max_timesteps = 150000
run_name = f"sac_{max_timesteps}_reward_shaping_"+task
eval_log_dir = os.path.join("eval_logs", run_name)
eval_callback = EvalCallback(eval_env, best_model_save_path=eval_log_dir,
                              log_path=eval_log_dir, eval_freq=max(500 // n_training_envs, 1),
                              n_eval_episodes=5, deterministic=True,
                              render=False)

In [ ]:
model.learn(total_timesteps=max_timesteps, callback=eval_callback)

/home/pc/repos/rl/rl-value-based/venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=500, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
New best mean reward!
Eval num_timesteps=1000, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=1500, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=2000, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=2500, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=3000, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=3500, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=4000, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=4500, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=5000, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=5500, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_ti

In [ ]:
# model.save(run_name)

In [17]:
model.load(f'eval_logs/{run_name}/best_model.zip')

In [23]:
for i in range(100):
	env_eval = make_env()
	obs_, _ = env_eval.reset()
	done = False
	ep_reward = 0

	while not done:
		action, _ = model.predict(obs_, deterministic=True)
		obs, reward, terminated, truncated, _ = env_eval.step(action)
		obs_ = obs
		done = terminated or truncated
		# Ignore reward shaping for evaluation
		# if reward < 1.0:
		# 	reward = 0.0 
		if done:
			ep_reward += reward
	print(f"Episode reward: {ep_reward}")

Episode reward: 0.5982661565294355
Episode reward: 0.5982661565294355


KeyboardInterrupt: 

In [ ]:
print("Action space:", env_eval.action_space)
print("Model action space:", model.action_space)

Action space: Box(-1.0, 1.0, (9,), float64)
Model action space: Box(-1.0, 1.0, (9,), float64)
